# 7. Wie gut lässt sich das Ergebnis eines Matches vorhersagen? Welches sind die wichtigsten Features für eine solche Prognose?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import plotly.express as px

In [2]:
# Laden der Daten
df = pd.read_csv("../../data/raw/atp_matches_till_2022.csv")

In [3]:
# Entfernen von nicht benötigten Spalten
unneeded_columns = [
    "score",
    "tourney_name",
    "winner_name",
    "loser_name",
    "minutes",
    "l_1stIn",
    "l_1stWon",
    "l_2ndWon",
    "l_ace",
    "l_svpt",
    "l_SvGms",
    "l_bpFaced",
    "l_df",
    "l_bpSaved",
    "w_1stIn",
    "w_1stWon",
    "w_2ndWon",
    "w_SvGms",
    "w_ace",
    "w_svpt",
    "w_bpFaced",
    "w_bpSaved",
    "w_df",
    "draw_size",
    "loser_entry",
    "winner_entry",
]
df.drop(columns=unneeded_columns, inplace=True)


In [4]:
# Behandlung fehlender Daten
essential_columns = [
    "winner_rank_points",
    "loser_rank_points",
    "winner_rank",
    "loser_rank",
    "surface",
]
df.dropna(subset=essential_columns, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
# Konvertierung der Datentypen
numeric_cols = [
    "winner_rank",
    "loser_rank",
    "winner_age",
    "loser_age",
    "winner_ht",
    "loser_ht",
]
df[numeric_cols] = df[numeric_cols].astype(float)

# Extrahieren von Datumskomponenten
df["tourney_year"] = df["tourney_date"].astype(str).str[:4].astype(int)
df["tourney_month"] = df["tourney_date"].astype(str).str[4:6].astype(int)
df.drop("tourney_date", axis=1, inplace=True)

In [6]:
# Spalten umbenennen
rename_map = {
    "loser_age": "p1_age",
    "loser_entry": "p1_entry",
    "loser_hand": "p1_hand",
    "loser_ht": "p1_ht",
    "loser_id": "p1_id",
    "loser_ioc": "p1_ioc",
    "loser_rank": "p1_rank",
    "loser_rank_points": "p1_rank_points",
    "loser_seed": "p1_seed",
    "winner_age": "p2_age",
    "winner_entry": "p2_entry",
    "winner_hand": "p2_hand",
    "winner_ht": "p2_ht",
    "winner_id": "p2_id",
    "winner_ioc": "p2_ioc",
    "winner_rank": "p2_rank",
    "winner_rank_points": "p2_rank_points",
    "winner_seed": "p2_seed",
}
df.rename(columns=rename_map, inplace=True)

In [7]:
# Erstellen gespiegelter Daten
mirrored_df = df.copy()
player_cols = [
    [
        f"p{i}_{attr}"
        for attr in ["age", "hand", "ht", "id", "ioc", "rank", "rank_points", "seed"]
    ]
    for i in [1, 2]
]
mirrored_df[player_cols[0] + player_cols[1]] = mirrored_df[
    player_cols[1] + player_cols[0]
]

In [8]:
# Label-Zuweisung
df["Win"], mirrored_df["Win"] = 0, 1
combined_df = (
    pd.concat([df, mirrored_df]).sample(frac=1, random_state=42).reset_index(drop=True)
)

In [9]:
# Kodierung kategorischer Daten
categorical_cols = [
    "p1_hand",
    "p2_hand",
    "p1_ioc",
    "p2_ioc",
    "surface",
    "tourney_level",
    "tourney_id",
    "round",
]
encoders = {
    col: LabelEncoder().fit(combined_df[col].astype(str)) for col in categorical_cols
}
for col, encoder in encoders.items():
    combined_df[col] = encoder.transform(combined_df[col].astype(str))

In [10]:
# Imputieren fehlender Werte
imputed_data = SimpleImputer().fit_transform(combined_df)
combined_df = pd.DataFrame(imputed_data, columns=combined_df.columns)

In [11]:
# Aufteilung des Datensatzes
y = combined_df["Win"]
X = combined_df.drop("Win", axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
# Trainieren des Modells
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [13]:
# Bewertung des Modells
train_accuracy = classifier.score(X_train, y_train)
test_accuracy = classifier.score(X_test, y_test)
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

Training Accuracy: 1.0
Test Accuracy: 0.6413746500627474


In [14]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    classifier, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

forest_importances = pd.Series(result.importances_mean, index=X.columns)

In [28]:
forest_importances_sorted = forest_importances.sort_values(ascending=False)

fig = px.bar(forest_importances_sorted)
fig.update_layout(
    title="Wichtigkeit der Merkmale",
    xaxis_title="Merkmale",
    yaxis_title="Wichtigkeit",
)

fig.update_traces(marker_color="#b2e061")
opacity_values = [1]*4 + [0.3]*(len(forest_importances_sorted)-4)
fig.update_traces(marker=dict(opacity=opacity_values))

fig.show()

(mit großen Abstand) Wichtigstes Feature: Spieler-Ranking und -Punkte: Höher gerankte Spieler gewinnen tendenziell häufiger.

Vorhandene Informationen:

- `Turnierinformationen`: Name, Oberfläche (Gras, Hartplatz, Sand, Teppich), Größe des Teilnehmerfeldes, Turnierlevel und -datum.
- `Matchdetails`: Spielnummer, IDs und Set-Ergebnisse der Spieler.
- `Spielerinformationen`: Setzplatz, Eintrittsmethode, Herkunftsland, Hand (Rechts-/Linkshänder), Größe, Geburtsdatum.
- `Spielstatistiken`: Erste und zweite Aufschlagprozente, Breakpoints, Asse, Doppelfehler usw.
- `Rankinginformationen`: ATP-Rang und Ranking-Punkte der Spieler.

Zusätzliche Informationen könnten hinzugezogen werden:

- `Spieler-Statistiken:` Dies beinhaltet aktuelle Weltranglistenpositionen, frühere Leistungen, Sieg-Niederlage-Statistiken, Performanz auf verschiedenen Belägen (Hartplatz, Rasen, Sand), etc.
- `Aktuelle Form`: Leistungen in den letzten Spielen oder Turnieren können ein Indikator für die aktuelle Form eines Spielers sein.
- `Head-to-Head-Bilanz`: Die vergangenen Aufeinandertreffen zwischen den Spielern können Aufschluss über psychologische Vorteile geben.
- `Physiologische und psychologische Faktoren`: Alter, Fitness, Verletzungen, mentale Stärke unter Druck, etc.
- `Umgebungsbedingungen`: Wetter, Belag, Höhe des Spielortes und Heimvorteil können ebenfalls eine Rolle spielen.